<a href="https://colab.research.google.com/github/thatishreshta/NeonWithCollab/blob/main/NeonWithColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install SQLAlchemy psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.7 MB/s eta 0:00:00


In [2]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

Paste STUDENT DB URL (hidden): ··········


In [3]:
# Example 1- Top 10 most expensive items overall (pc+laptop+printer)
with engine.connect() as conn:
    sql = """
    WITH all_items AS (
      SELECT 'pc'::text AS kind, model, price FROM pc
      UNION ALL
      SELECT 'laptop', model, price FROM laptop
      UNION ALL
      SELECT 'printer', model, price FROM printer
    )
    SELECT a.kind, p.maker, a.model, a.price
    FROM all_items a
    JOIN product p USING (model)
    ORDER BY a.price DESC NULLS LAST
    LIMIT 10;
    """
    df = pd.read_sql_query(text(sql), conn)
df

,kind,maker,model,price
0,laptop,E,2001,3673
1,laptop,A,2005,2500
2,laptop,G,2010,2300
3,pc,A,1001,2114
4,laptop,A,2006,1700
5,laptop,B,2007,1429
6,laptop,A,2004,1150
7,pc,B,1006,1049
8,pc,A,1002,995
9,pc,E,1011,959


In [4]:
# Example2 - Avg/Min/Max per category
with engine.connect() as conn:
    sql = """
    SELECT 'pc' AS kind, ROUND(AVG(price),2) avg_price, MIN(price) min_price, MAX(price) max_price FROM pc
    UNION ALL
    SELECT 'laptop', ROUND(AVG(price),2), MIN(price), MAX(price) FROM laptop
    UNION ALL
    SELECT 'printer', ROUND(AVG(price),2), MIN(price), MAX(price) FROM printer;
    """
    df = pd.read_sql_query(text(sql), conn)
df

,kind,avg_price,min_price,max_price
0,pc,827.08,478,2114
1,laptop,1583.00,549,3673
2,printer,253.86,99,899


In [8]:
# Example3 - Printer type distribution and avg price
with engine.connect() as conn:
    sql = """
    SELECT pr.type AS printer_type, COUNT(*) AS cnt,
           ROUND(AVG(pr.price),2) AS avg_price
    FROM printer pr
    GROUP BY pr.type
    ORDER BY cnt DESC, printer_type;
    """
    df = pd.read_sql_query(text(sql), conn)
df

,printer_type,cnt,avg_price
0,laser,4,364.50
1,ink-jet,3,106.33


In [9]:
#Example3- Makers who produce both PCs and printers
with engine.connect() as conn:
    makers_sql = """
    SELECT maker
    FROM product
    WHERE type IN ('pc','printer')
    GROUP BY maker
    HAVING COUNT(DISTINCT type) = 2
    ORDER BY maker;
    """
    makers_df = pd.read_sql_query(text(makers_sql), conn)

print("Makers producing both PCs and printers:")
display(makers_df)

Makers producing both PCs and printers:


,maker
0,D
1,E


In [10]:
#Example5- Correlation between laptop speed and price
with engine.connect() as conn:
    sql = """
    SELECT CORR(price, speed) AS corr_price_speed
    FROM laptop
    WHERE price IS NOT NULL AND speed IS NOT NULL;
    """
    df = pd.read_sql_query(text(sql), conn)
df

,corr_price_speed
0,0.706414


In [11]:
#Example6- Top laptop (largest screen, then highest price) per maker
with engine.connect() as conn:
    top_screens_sql = """
    SELECT DISTINCT ON (p.maker)
           p.maker, l.model, l.screen
    FROM product p
    JOIN laptop l USING (model)
    ORDER BY p.maker, l.screen DESC NULLS LAST, l.price DESC NULLS LAST;
    """
    top_screens_df = pd.read_sql_query(text(top_screens_sql), conn)

display(top_screens_df)

,maker,model,screen
0,A,2005,17.0
1,B,2007,13.3
2,E,2001,20.1
3,F,2008,15.4
4,G,2010,15.4
